<center>
<img src="../../img/beeline_data_school_logo.png">
# Школа данных "Билайн"
#### Интенсивный курс по изучению машинного обучения и анализа данных
<img src="../../img/beeline_logo.jpg" height="240" width="240">
## Автор материала: преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

## Домашнее задание 2. Часть 1. Построение дерева решений

**Цель задания - "на пальцах", с помощью игрушечных задач классификации разобраться в том, как работают деревья решений. Напомню, само по себе дерево решений - довольно слабый алгоритм, но основанные на нем алгоритмы случайного леса и градиентного бустинга - пожалуй, лучшее, что есть на сегодняшний день. Поэтому разобраться в том, как работет дерево решений, полезно.**

**Рассмотрим 3 игрушечных задачи бинарной классификации (символ $\tau$ вместо знака вопроса обозначает неизвестные метки целевого класса тестовых объектов):**
<img src='../../img/toy_clf_task1.png'>
<img src='../../img/toy_clf_task2.png'>
<img src='../../img/toy_clf_task3.png'>

In [6]:
import pandas as pd
import graphviz as gv
from sklearn.preprocessing import LabelEncoder
import math

In [7]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    f = list( set(train.keys()) & set(test.keys()))
    return train[f], test[f]

### Игрушечный набор данных №1. "Служба знакомств"

In [8]:
fn_1 = ['Возраст', 'Доход','Внешность','Образование'] # знакомства
fn_2 = ['Цвет', 'Тип', 'Производство', 'Повреждения'] # угон
fn_3 = ['Пол', 'Возраст', 'Образование', 'Зарпалата'] # кредит

**Обучающая выборка**

In [9]:
df1_train = {}
df1_train['Возраст'] = ['молодой','средний','пожилой','средний','молодой','средний','пожилой'] 
df1_train['Доход'] = ['высокий','высокий','высокий','средний','низкий','высокий','средний']
df1_train['Внешность'] = ['приятная','приятная','приятная','приятная','приятная','отталкивающая','приятная']
df1_train['Образование'] = ['специальное','высшее','высшее','высшее','специальное','высшее','среднее']
df1_train['Составит пару'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

df1_train = create_df(df1_train, fn_1)

**Тестовая выборка**

In [10]:
df1_test = {}
df1_test['Возраст'] = ['молодой','средний','пожилой'] 
df1_test['Доход'] = ['средний','высокий','низкий']
df1_test['Внешность'] = ['приятная','отталкивающая','приятная']
df1_test['Образование'] = ['высшее','специальное','высшее']
df1_test = create_df(df1_test, fn_1)
df1_test

,Возраст_молодой,Возраст_пожилой,Возраст_средний,Доход_высокий,Доход_низкий,Доход_средний,Внешность_отталкивающая,Внешность_приятная,Образование_высшее,Образование_специальное
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [11]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_1 = df1_train['Составит пару']
df1_train, df1_test = intersect_features(train = df1_train, test = df1_test)
df1_train

,Доход_средний,Внешность_приятная,Возраст_молодой,Образование_специальное,Доход_низкий,Образование_высшее,Возраст_пожилой,Доход_высокий,Возраст_средний,Внешность_отталкивающая
0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
6,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
pd.concat([df1_train, y_1], axis=1)

,Доход_средний,Внешность_приятная,Возраст_молодой,Образование_специальное,Доход_низкий,Образование_высшее,Возраст_пожилой,Доход_высокий,Возраст_средний,Внешность_отталкивающая,Составит пару
0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
4,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1
6,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1


In [15]:
df1_test

,Доход_средний,Внешность_приятная,Возраст_молодой,Образование_специальное,Доход_низкий,Образование_высшее,Возраст_пожилой,Доход_высокий,Возраст_средний,Внешность_отталкивающая
0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


### Игрушечный набор данных №2. "Автоугон"

In [18]:
# обучение

df2_train = {}
df2_train['Цвет'] = ['Красный','Желтый','Желтый','Красный','Желтый','Желтый','Красный'] 
df2_train['Тип'] = ['Спортивный','Спортивный','Джип','Спортивный','Спортивный','Джип','Джип']
df2_train['Производство'] = ['США','Япония','Япония','Япония','США','США','Япония']
df2_train['Повреждения'] = ['нет','нет','нет','есть','есть','нет','есть']
df2_train['Угоняют'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

3.314467420322094

In [19]:
# тест
df2_test = {}
df2_test['Цвет'] = ['Зеленый','Красный','Черный'] 
df2_test['Тип'] = ['Спортивный','Спортивный','Джип']
df2_test['Производство'] = ['США','Германия','Япония']
df2_test['Повреждения'] = ['нет','нет','нет']

df2_test = create_df(df2_test, fn_2)
df2_test

,Цвет_Зеленый,Цвет_Красный,Цвет_Черный,Тип_Джип,Тип_Спортивный,Производство_Германия,Производство_США,Производство_Япония,Повреждения_нет
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0


In [20]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_2 = df2_train['Угоняют']
df2_train, df2_test = intersect_features(train = df2_train, test = df2_test)
df2_train

,Производство_Япония,Тип_Джип,Тип_Спортивный,Повреждения_нет,Цвет_Красный,Производство_США
0,0.0,0.0,1.0,1.0,1.0,1.0
1,1.0,0.0,1.0,1.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0
5,0.0,1.0,0.0,1.0,0.0,1.0
6,1.0,1.0,0.0,0.0,1.0,0.0


In [21]:
df2_test

,Производство_Япония,Тип_Джип,Тип_Спортивный,Повреждения_нет,Цвет_Красный,Производство_США
0,0.0,0.0,1.0,1.0,0.0,1.0
1,0.0,0.0,1.0,1.0,1.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0


### Игрушечный набор данных №3. "Выдача кредита"

In [22]:
# обучение

df3_train = {}
df3_train['Пол'] = LabelEncoder().fit_transform(['М','Ж','Ж','М','М','Ж','Ж'])
df3_train['Возраст'] = ['молодой','средний','средний','пожилой','молодой','средний','пожилой']
df3_train['Образование'] = ['высшее','специальное','высшее','высшее','высшее','среднее','специальное']
df3_train['Зарпалата'] = ['высокая','высокая','средняя','высокая','низкая','средняя','средняя']
df3_train['Выдать кредит'] = LabelEncoder().fit_transform(['+','+','+','+','-','-','-'])

df3_train = create_df(df3_train, fn_3)
df3_train['Пол'] = LabelEncoder().fit_transform(['М','Ж','Ж','М','М','Ж','Ж'])
df3_train


,Выдать кредит,Возраст_молодой,Возраст_пожилой,Возраст_средний,Образование_высшее,Образование_специальное,Образование_среднее,Зарпалата_высокая,Зарпалата_низкая,Зарпалата_средняя,Пол
0,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0
2,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0
3,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
4,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
5,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0
6,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0


In [23]:
# тест

df3_test = {}
df3_test['Пол'] =  LabelEncoder().fit_transform(['Ж','Ж','М'])
df3_test['Возраст'] = ['молодой','пожилой','средний']
df3_test['Образование'] = ['специальное','высшее','специальное']
df3_test['Зарпалата'] = ['высокая','средняя','средняя']

df3_test = create_df(df3_test, fn_3)
df3_test['Пол'] =  LabelEncoder().fit_transform(['Ж','Ж','М'])
df3_test

,Возраст_молодой,Возраст_пожилой,Возраст_средний,Образование_высшее,Образование_специальное,Зарпалата_высокая,Зарпалата_средняя,Пол
0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1


In [24]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y_3 = df3_train['Выдать кредит']
df3_train, df3_test = intersect_features(train = df3_train, test = df3_test)
df3_train

,Возраст_молодой,Образование_специальное,Зарпалата_средняя,Зарпалата_высокая,Образование_высшее,Возраст_пожилой,Возраст_средний,Пол
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0
3,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
6,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0


In [25]:
df3_test

,Возраст_молодой,Образование_специальное,Зарпалата_средняя,Зарпалата_высокая,Образование_высшее,Возраст_пожилой,Возраст_средний,Пол
0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0
1,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1


## Функции для расчета энтропии и прироста информации

In [26]:
# данные для тестирования: 9 синих шариков и 11 желтых
b = [1 for i in range(9)] + [0 for i in range(11)]

# два разбиения
b1 = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
b2 = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых


In [27]:
# расчет энтропии Шеннона
def entropy(a_list):
    H = 0
    for n in set(a_list):
        p = a_list.count(n)*1./len(a_list)
        H = H + (p)* math.log(p,2)
    return -H

In [28]:
print entropy([1,0,0,0,0,0,0]) # 1 синий и 6 желтых
print entropy(b)
print entropy([1,2,3,4,5,6]) # энтропия игральной кости с несмещенным центром тяжести

0.591672778582
0.992774453988
2.58496250072


In [29]:
# расчет прироста информации
def information_gain(root, left, right):
    GI = entropy(root) - len(left)*1./len(root)*entropy(left) - len(right)*1./len(root)*entropy(right)
    return GI    

In [30]:
print(information_gain(b,b1,b2))

0.160885188414


In [31]:
# функция для расчета прироста информации при разбиении набора данных 
def split(X,y):
    ''' Выводит прирост информации при разбиении по каждому признаку'''
    MaxGI = 0;
    root = pd.concat([X, y_1], axis=1)
    shape = len(root.columns)

    for col in range(shape-1):
        left = root[root.iloc[:,col]==1]
        right = root[root.iloc[:,col]==0]

        GI = information_gain(root.iloc[:,shape-1].tolist(),\
                         left.iloc[:,shape-1].tolist(),\
                         right.iloc[:,shape-1].tolist())
        print (root.columns[col] + ' - entropy:  ' +str(GI))
        if MaxGI < GI:
            MaxGI = GI
            MaxCol = col

    print ('\n' + root.columns[MaxCol] + ' - entropy:  ' +str(MaxGI))
    y_l = root[root.iloc[:, MaxCol] == 1].iloc[:,shape-1]
    y_r = root[root.iloc[:, MaxCol] == 0].iloc[:,shape-1]
    left = X[X.iloc[:, MaxCol] == 1]
    right = X[X.iloc[:, MaxCol] == 0]
    return   left,y_l,right,y_r,root.columns[MaxCol], MaxGI

## Постройте деревья решений для трех наборов данных. 

**Рисовать деревья можно от руки. Дополнительно (для желающих) - отрисовка деревьев, код для построения всего дерева.**

In [1]:
#Построение дерева решений
def buildTree(df_train, y, graph, ParenNode=0):
    if entropy(y) == 0:
        CurrentNode = ParenNode + 1
        graph.edge(str(CurrentNode,ParenNode))
    else:
        left,y_l,right,y_r,paramSTR,MaxGi = split(df_train, y)
        CurrentNode = ParenNode + 1
        graph.node(str(CurrentNode),{'label' : paramSTR + '\n' + ' entropy: ' + MaxGi})
        if CurrentNode!=1:
            graph.edge(str(CurrentNode,ParenNode))
        ParenNode = CurrentNode
        buildTree(left,y_l,ParenNode=CurrentNode,graph=graph)
        buildTree(right,y_r,ParenNode=CurrentNode,graph=graph)
    return graph

In [10]:
graph = gv.Graph(format='svg')
buildTree(df_train, y, graph, ParenNode=0)
graph.render('img/graph/')

NameError: name 'df_train' is not defined

## Постройте деревья решений для трех наборов данных с помощью sklearn

**Отобразите деревья с помощью graphviz.**

## "Служба знакомств"

<img src="Data/df1.png"> 

## "Автоугон"

<img src="Data/df2.png"> 

## "Выдача кредита"

<img src="Data/df3.png"> 